In [2]:
import pandas as pd
import datetime as dt
import footballData as fd
import numpy as np
import random
import matplotlib.pyplot as plt
import datetime
import time
import Elo as ELO
import seaborn as sns

In [3]:
df1888=pd.read_csv('englandthesis.csv')

/Users/andrewpuopolo/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
startdate=1958
fulldf=df1888[df1888['Season']>=startdate]
fulldf=fulldf.sort_values('Date')
fulldf=fulldf.reset_index()
fulldf=fulldf.replace('Milton Keynes Dons', 'Wimbledon')

In [5]:
def CalendarYear(Date):
    return int(Date.split('-')[0])
CalendarYear(fulldf['Date'][0])
def Month(Date):
    return int(Date.split('-')[1])
fulldf['CalendarYear']=fulldf.apply(lambda row: CalendarYear(row['Date']), axis=1)
fulldf['Month']=fulldf.apply(lambda row: Month(row['Date']), axis=1)

In [6]:
import copy
from copy import deepcopy


def E(r, rj, Home):
    return 1/(1+10**(-1*(r-rj+Home)/400))

def Res(HomeG, AwayG):
    if(HomeG>AwayG):
        return 1
    elif(AwayG>HomeG):
        return 0
    else:
        return 0.5

def lik(E, R):
    if(R==1):
        return np.log(E)
    elif (R==.5):
        return np.log(np.sqrt(E*(1-E)))
    elif (R==0):
        return np.log(1-E)

initdictionary={}
initvalues=[1500,1500,1500,1500]
initialrows=[]


#This needs to be rewritten so we don't have to run it over an dover again
initdictionary={}
initvalues=[1500,1500,1500,1500]
initialrows=[]
#This needs to be rewritten so we don't have to run it over an dover again
for i in range(len(fulldf['home'].unique())):
    Team=fulldf['home'].unique()[i]
    Teamdf=fulldf[fulldf['home']==Team].reset_index()
    Tier=int(Teamdf['tier'][0])
    initialrow=[Team, Tier]
    initialrows.append(initialrow)
#Needs to be rewritten to optimize using BuildEloDict Function
for a in range(len(initialrows)):
    initdictionary[initialrows[a][0]]=initvalues[initialrows[a][1]-1]
initdictionary

{'AFC Bournemouth': 1500,
 'AFC Wimbledon': 1500,
 'Accrington': 1500,
 'Accrington Stanley': 1500,
 'Aldershot': 1500,
 'Arsenal': 1500,
 'Aston Villa': 1500,
 'Barnet': 1500,
 'Barnsley': 1500,
 'Barrow': 1500,
 'Birmingham City': 1500,
 'Blackburn Rovers': 1500,
 'Blackpool': 1500,
 'Bolton Wanderers': 1500,
 'Boston United': 1500,
 'Bradford City': 1500,
 'Bradford Park Avenue': 1500,
 'Brentford': 1500,
 'Brighton & Hove Albion': 1500,
 'Bristol City': 1500,
 'Bristol Rovers': 1500,
 'Burnley': 1500,
 'Burton Albion': 1500,
 'Bury': 1500,
 'Cambridge United': 1500,
 'Cardiff City': 1500,
 'Carlisle United': 1500,
 'Charlton Athletic': 1500,
 'Chelsea': 1500,
 'Cheltenham': 1500,
 'Chester': 1500,
 'Chesterfield': 1500,
 'Colchester United': 1500,
 'Coventry City': 1500,
 'Crawley Town': 1500,
 'Crewe Alexandra': 1500,
 'Crystal Palace': 1500,
 'Dagenham and Redbridge': 1500,
 'Darlington': 1500,
 'Derby County': 1500,
 'Doncaster Rovers': 1500,
 'Everton': 1500,
 'Exeter City': 15

In [7]:
import datetime
from datetime import datetime
def DT(Date):
    return datetime.strptime(Date, '%Y-%m-%d')
fulldf['Date']=fulldf.apply(lambda row: DT(row['Date']), axis=1)

def to_integer(current_time):
    return (current_time-fulldf['Date'][0]).days
fulldf['Day#']=fulldf.apply(lambda row: to_integer(row['Date']), axis=1)
fulldf

,index,Unnamed: 0,Date,Season,home,visitor,FT,hgoal,vgoal,division,tier,totgoal,goaldif,result,CalendarYear,Month,Day#
0,75629,75630,1958-08-23,1958,Swindon Town,Wrexham,1-0,1,0,3,3,1,1,H,1958,8,0
1,75423,75424,1958-08-23,1958,Notts County,Accrington Stanley,1-1,1,1,3,3,2,0,D,1958,8,0
2,75745,75746,1958-08-23,1958,Carlisle United,Aldershot,1-0,1,0,4,4,1,1,H,1958,8,0
3,75743,75744,1958-08-23,1958,Bradford Park Avenue,Workington,3-2,3,2,4,4,5,1,H,1958,8,0
4,74746,74747,1958-08-23,1958,Cardiff City,Barnsley,0-1,0,1,2,2,1,-1,A,1958,8,0
5,75708,75709,1958-08-23,1958,Barrow,Gateshead,0-3,0,3,4,4,3,-3,A,1958,8,0
6,74337,74338,1958-08-23,1958,Burnley,Manchester City,3-4,3,4,1,1,7,-1,A,1958,8,0
7,74783,74784,1958-08-23,1958,Charlton Athletic,Sheffield United,1-1,1,1,2,2,2,0,D,1958,8,0
8,74827,74828,1958-08-23,1958,Fulham,Stoke City,6-1,6,1,2,2,7,5,H,1958,8,0
9,75637,75638,1958-08-23,1958,Tranmere Rovers,Doncaster Rovers,3-0,3,0,3,3,3,3,H,1958,8,0


In [8]:
fulldf

,index,Unnamed: 0,Date,Season,home,visitor,FT,hgoal,vgoal,division,tier,totgoal,goaldif,result,CalendarYear,Month,Day#
0,75629,75630,1958-08-23,1958,Swindon Town,Wrexham,1-0,1,0,3,3,1,1,H,1958,8,0
1,75423,75424,1958-08-23,1958,Notts County,Accrington Stanley,1-1,1,1,3,3,2,0,D,1958,8,0
2,75745,75746,1958-08-23,1958,Carlisle United,Aldershot,1-0,1,0,4,4,1,1,H,1958,8,0
3,75743,75744,1958-08-23,1958,Bradford Park Avenue,Workington,3-2,3,2,4,4,5,1,H,1958,8,0
4,74746,74747,1958-08-23,1958,Cardiff City,Barnsley,0-1,0,1,2,2,1,-1,A,1958,8,0
5,75708,75709,1958-08-23,1958,Barrow,Gateshead,0-3,0,3,4,4,3,-3,A,1958,8,0
6,74337,74338,1958-08-23,1958,Burnley,Manchester City,3-4,3,4,1,1,7,-1,A,1958,8,0
7,74783,74784,1958-08-23,1958,Charlton Athletic,Sheffield United,1-1,1,1,2,2,2,0,D,1958,8,0
8,74827,74828,1958-08-23,1958,Fulham,Stoke City,6-1,6,1,2,2,7,5,H,1958,8,0
9,75637,75638,1958-08-23,1958,Tranmere Rovers,Doncaster Rovers,3-0,3,0,3,3,3,3,H,1958,8,0


In [38]:
import time


def lik(E, R):
    if(R==1):
        return np.log(E)
    elif (R==.5):
        return np.log(np.sqrt(E*(1-E)))
    elif (R==0):
        return np.log(1-E)
def Eloperiods(params):
    ratingperiodlength=7.0
    k=params['k']
    h=80
    hk=params['hk']
    #minrd=params['minrd']
    curtime=time.time()
    rows=len(fulldf)
    logliks=0.0
    hburn=9
    burn=41
    startyear=fulldf['Season'].values[0]
    startday=fulldf['Season']
    Elodictionary2=copy.deepcopy(initdictionary)
    currentyear=fulldf['Season'].values[0]
    startdate=fulldf['Date'][0]
    GameCounter=0.
    Rs=0.
    Es=0.
    year=copy.deepcopy(startyear)
    totaltime=fulldf['Day#'][rows-1]-fulldf['Day#'][0]
    day=0.
    games=0
    while day<=totaltime:
        #print(Elodictionary2)
        ratingperioddf=fulldf[(fulldf['Day#'] >=day) & (fulldf['Day#']<day+ratingperiodlength)].reset_index()
        if len(ratingperioddf!=0):

            Rs=0.
            Es=0.

            for ind, row in ratingperioddf.iterrows():
                Home=row['home']
                Away=row['visitor']
                HomeGoals=row['hgoal']
                AwayGoals=row['vgoal']
                year=row['Season']
                date=row['Date']
                tier=row['tier']
                HomeRating=Elodictionary2[Home]
                AwayRating=Elodictionary2[Away]
                Expect=E(HomeRating, AwayRating, h)
                HRes=Res(HomeGoals, AwayGoals)
                PE=(HRes-Expect)*k
                Elodictionary2[Home]=HomeRating+PE
                Elodictionary2[Away]=AwayRating-PE
                Rs+=HRes
                Es+=Expect
                if(year>startyear+burn):
                    logliks+=lik(Expect, HRes)
                    games+=1
                #print(Home,Away, HomeRating, AwayRating, PE, HRes, HomeGoals, AwayGoals)
                #print(Elodictionary2['Home'], Elodictionary2['Away'], h)
                    
            
            if(year>startyear+hburn):
                h+=hk*(Rs-Es)
        day+=ratingperiodlength
    print(params ,logliks,Elodictionary2['Tottenham Hotspur'], h,games,time.time()-curtime)
    return -1*logliks

In [10]:
Eloperiods(best)

NameError: name 'best' is not defined

In [11]:
best={'hk': 0.2742633382088742, 'k': 13.797215454070143}
Eloperiods(best)

{'hk': 0.2742633382088742, 'k': 13.797215454070143} -1461.21787538 1960.1443789944783 29.780027866363525


[0, 0]

In [56]:
columns=['Year', 'Home Results']
Homedf=pd.DataFrame(Hres, columns=columns)
Homedf=Homedf[Homedf['Year']>1968].reset_index()
plt.scatter(Homedf['Year'], Homedf['Home Results'], color='green')
plt.ylabel('Average Home Result', fontsize=14)
plt.xlabel('Year', fontsize=14)
plt.title('Average Home Result Over Time', fontsize=16)
plt.savefig('HomeResTime.png')

In [ ]:
from hyperopt import hp
space = {
    #'ratingperiodlength': hp.quniform('ratingperiodlength', 2, 30, 1),
    'c': hp.uniform('c', 0, 40),
    #'h': hp.uniform('h', 50, 150),
    #get rid of this for now
    #'minrd': hp.uniform('minrd', 0, 60),
    'hk': hp.uniform('hk', 0, .25),
    'maxrd': hp.uniform('maxrd', 100, 350)
}

from hyperopt import tpe
# Algorithm
tpe_algorithm = tpe.suggest

from hyperopt import Trials
# Trials object to track progress
bayes_trials = Trials()

from hyperopt import fmin

MAX_EVALS = 1000



space = {
        #'ratingperiodlength': hp.quniform('ratingperiodlength', 2, 30, 1),
        'k': hp.uniform('k', 0, 100),
        #'h': hp.uniform('h', 50, 150),
        #get rid of this for now
        #'minrd': hp.uniform('minrd', 0, 60),
        'hk': hp.uniform('hk', 0, 1),
    }

from hyperopt import tpe
    # Algorithm
tpe_algorithm = tpe.suggest

from hyperopt import Trials
    # Trials object to track progress
bayes_trials = Trials()

from hyperopt import fmin

MAX_EVALS = 2500

best2000 = fmin(fn = Eloperiods, space = space, algo = tpe.suggest, 
        max_evals = MAX_EVALS, trials = bayes_trials)


{'hk': 0.3096510862810854, 'k': 53.22097350019448} -23319.8011598 2206.4761011436945 54.27919949851894 34612 24.37980008125305
{'hk': 0.1259301831324534, 'k': 18.100926617559686} -22879.9341538 2013.4127812780603 48.65642927564436 34612 24.800710201263428
{'hk': 0.3833845699828775, 'k': 75.71971386218472} -23728.6803454 2262.086776007581 57.95190146827593 34612 25.94700312614441
{'hk': 0.35371438428262003, 'k': 88.8864495570781} -23992.6082436 2287.727363262973 57.922269173975586 34612 27.052573919296265
{'hk': 0.8006989662462948, 'k': 76.9684497993847} -23755.0252487 2265.9009927111797 68.00642417737875 34612 26.20372986793518
{'hk': 0.22144536019899763, 'k': 21.103702407195478} -22896.0864702 2042.7681078933301 49.656681371388 34612 26.80460023880005
{'hk': 0.22643460886060462, 'k': 50.063507068972115} -23268.0510835 2196.793739549522 51.86213311423499 34612 26.406502962112427
{'hk': 0.08435840829773678, 'k': 9.297100301927719} -22891.5449466 1886.9176725809323 49.26176248598211 3461

{'hk': 0.44697407754273455, 'k': 13.201758248210917} -22868.6394394 1951.629375456923 54.543673439538395 34612 22.7486572265625
{'hk': 0.5231242197749314, 'k': 16.597809258394722} -22872.8349975 1995.9215959472767 56.739322495777685 34612 24.113330125808716
{'hk': 0.9664570682410819, 'k': 25.7740985517176} -22939.3116611 2081.330016723334 65.43206841589003 34612 21.107656955718994
{'hk': 0.3087576705098166, 'k': 29.380514765253235} -22973.979285 2105.2204899723893 52.518767763409954 34612 22.569298028945923
{'hk': 0.18707676401841553, 'k': 15.586475941521911} -22869.7856845 1983.9067850279678 48.55867816654326 34612 22.34628415107727
{'hk': 0.6475316402219667, 'k': 20.75288044668099} -22894.8332686 2039.4941829279098 59.83939289223155 34612 24.153857946395874
{'hk': 0.5914325573786393, 'k': 0.9220751323398861} -23233.3524681 1580.57508258583 55.310481949675854 34612 25.678019046783447
{'hk': 0.44719600022107264, 'k': 9.4445080063851} -22886.3806896 1888.9879527269627 54.07512532371638 

In [28]:
 Eloperiods(best1st)

{'hk': 0.02527631700368402, 'k': 13.089328355638918} -3984.63175072 1953.5583064149698 62.858942215504136 6460 24.437364101409912


3984.6317507176586

In [ ]:
nulldf=fulldf[fulldf['Season']>1983].reset_index()

In [ ]:
nullpred=1/(1+10**(-.2))
nullpred

In [ ]:
nullliks=0
for i in range(len(nulldf)):
    if(nulldf['hgoal'][i]>nulldf['vgoal'][i]):
        
        nullliks+=np.log(nullpred)
    elif(nulldf['hgoal'][i]<nulldf['vgoal'][i]):
        nullliks+=np.log(1-nullpred)
    else:
        nullliks+=np.log(np.sqrt(nullpred*(1-nullpred)))
nullliks

In [ ]:
np.log(np.sqrt(nullpred*(1-nullpred)))

In [ ]:
np.log(1-nullpred)

In [41]:
teamsforplot=Eloperiods(best)

{'hk': 0.2742633382088742, 'k': 13.797215454070143} -44210.7270572 1960.1077540754864 19.827125072479248


In [44]:
columns=['Date', 'Manchester United', 'Portsmouth', 'Manchester City']
teamsdf=pd.DataFrame(teamsforplot, columns=columns)


In [47]:
teamsdf['Date'][0].year

1958

In [50]:
teamsdf['Year']=teamsdf['Date'].dt.year

In [52]:
teamsdfplot=teamsdf[teamsdf['Year']>1992].reset_index()

In [77]:
plt.plot(teamsdfplot['Date'], teamsdfplot['Manchester United'], color='red')
plt.plot(teamsdfplot['Date'], teamsdfplot['Manchester City'], color='skyblue')
plt.plot(teamsdfplot['Date'], teamsdfplot['Portsmouth'], color='blue')
plt.xlabel('Year', fontsize=14)
plt.ylabel('Elo Rating', fontsize=14)
plt.legend(('Manchester United', 'Manchester City', 'Portsmouth'), fontsize=11)
plt.title('Elo Rating Of 3 Major English Clubs', fontsize=16)
plt.savefig('3TeamElo.png')

In [58]:
teamsdfplot

,index,Date,Manchester United,Portsmouth,Manchester City,Year
0,1793,1993-01-08,1818.989303,1607.366748,1761.622551,1993
1,1794,1993-01-15,1822.598446,1610.877403,1769.889538,1993
2,1795,1993-01-22,1829.552712,1606.405435,1762.223385,1993
3,1796,1993-01-29,1832.885820,1610.420025,1768.871996,1993
4,1797,1993-02-05,1825.010818,1611.171007,1772.525514,1993
5,1798,1993-02-10,1828.367175,1607.274281,1773.507331,1993
6,1799,1993-02-19,1828.367175,1613.384390,1773.507331,1993
7,1800,1993-02-26,1831.794414,1611.475982,1759.181325,1993
8,1801,1993-03-05,1834.099192,1606.053200,1767.012062,1993
9,1802,1993-03-12,1832.401221,1616.135837,1771.235542,1993


In [15]:
years=Eloperiods(best)

{'hk': 0.2742633382088742, 'k': 13.797215454070143} -44210.7270572 1960.1077540754864 20.75425100326538


In [341]:
best

{'hk': 0.2742633382088742, 'k': 13.797215454070143}

In [16]:
rows=[]
for i in range(len(years)):
    row=[years[i][0], np.mean(years[i][1]), np.std(years[i][1]),
        np.mean(years[i][2]), np.std(years[i][2]),
         np.mean(years[i][3]), np.std(years[i][3]),
         np.mean(years[i][4]), np.std(years[i][4])]
    rows.append(row)

In [17]:
columns=['Year', 'Premier League', 'PremStd', 'Championship', 'ChampStd', 'League One', 'L1Std', 'League Two', 'L2Std']

In [18]:
pd.DataFrame(rows, columns=columns)

,Year,Premier League,PremStd,Championship,ChampStd,League One,L1Std,League Two,L2Std
0,1958,1515.661737,42.390806,1498.174101,45.297324,1510.278942,42.281877,1477.038206,40.425672
1,1959,1524.923476,43.739517,1500.624259,46.074763,1515.380063,33.032479,1464.624502,43.463868
2,1960,1536.465058,54.366218,1509.242929,50.140425,1513.444680,43.424912,1448.079653,32.351849
3,1961,1548.184666,47.828906,1509.181837,37.886550,1513.540818,40.395669,1443.755537,44.073864
4,1962,1555.238838,46.830447,1515.018155,40.682027,1511.070650,31.283038,1434.409424,46.404502
5,1963,1568.680191,48.882742,1511.119753,35.674708,1510.773920,28.947415,1425.958447,39.549853
6,1964,1578.582276,48.238542,1513.902921,39.441879,1511.983723,39.478963,1413.120496,52.550015
7,1965,1586.940470,49.321419,1524.183351,49.219125,1500.170605,35.607188,1407.848209,42.284725
8,1966,1600.814936,51.066806,1520.979145,41.754579,1495.090835,31.337533,1403.146908,36.066121
9,1967,1611.278332,52.139963,1518.484268,30.408830,1492.778761,30.083142,1398.154506,50.012198


In [19]:
leagueaverages=pd.DataFrame(rows, columns=columns)

In [20]:
cutleagueavg=leagueaverages[leagueaverages['Year']>1983].reset_index()

In [21]:
cutleagueavg['League One']-cutleagueavg['League Two']

0     137.014449
1     129.323573
2     118.887399
3     114.128071
4     106.176375
5     102.816893
6      96.666360
7     119.549179
8     118.924044
9     128.907978
10    145.353247
11    145.121266
12    135.941951
13    136.485707
14    118.535421
15    114.469349
16    114.994481
17    112.883823
18    119.690715
19    115.893832
20    118.780603
21    103.787871
22     98.500827
23    110.570430
24    111.271443
25    102.224149
26     89.067730
27     97.687878
28    108.126153
29    109.061390
30    110.223679
31    116.599015
dtype: float64

In [38]:
cutleagueavg['League One']-cutleagueavg['League Two']

0     137.014449
1     129.323573
2     118.887399
3     114.128071
4     106.176375
5     102.816893
6      96.666360
7     119.549179
8     118.924044
9     128.907978
10    145.353247
11    145.121266
12    135.941951
13    136.485707
14    118.535421
15    114.469349
16    114.994481
17    112.883823
18    119.690715
19    115.893832
20    118.780603
21    103.787871
22     98.500827
23    110.570430
24    111.271443
25    102.224149
26     89.067730
27     97.687878
28    108.126153
29    109.061390
30    110.223679
31    116.599015
dtype: float64

In [35]:
plt.plot(cutleagueavg['Year'], cutleagueavg['PremStd'], color='red', linewidth=0.5)
plt.plot(cutleagueavg['Year'], cutleagueavg['ChampStd'], color='blue', linewidth=0.5)
plt.plot(cutleagueavg['Year'], cutleagueavg['L1Std'], color='green', linewidth=0.5)
plt.plot(cutleagueavg['Year'], cutleagueavg['L2Std'], color='brown', linewidth=0.5)
plt.legend(('Premier League', 'Championship', 'League One', 'League Two'))
plt.xlabel('Year', fontsize=14)
plt.ylabel('Std Dev', fontsize=14)
plt.title('Standard Deviation Of Elo Scores By Division', fontsize=16)
plt.savefig('DivStdDev.png')
#plt.show()

In [32]:
fig, ax = plt.subplots(1,1,figsize=(8,5))
#lowerbound=cutleagueavg['PremMean']-cutleagueavg['PremStd']
ax.plot( cutleagueavg['Year'], cutleagueavg['Premier League'], color='red')
ax.fill_between(cutleagueavg['Year'], cutleagueavg['Premier League']-cutleagueavg['PremStd'],cutleagueavg['Premier League']+cutleagueavg['PremStd'], facecolor='red', alpha=.1)
ax.plot( cutleagueavg['Year'], cutleagueavg['Championship'], color='blue')
ax.fill_between(cutleagueavg['Year'], cutleagueavg['Championship']-cutleagueavg['ChampStd'],cutleagueavg['Championship']+cutleagueavg['ChampStd'], facecolor='blue', alpha=0.1)
ax.plot( cutleagueavg['Year'], cutleagueavg['League One'], color='green')
ax.fill_between(cutleagueavg['Year'], cutleagueavg['League One']-cutleagueavg['L1Std'],cutleagueavg['League One']+cutleagueavg['L1Std'], facecolor='green', alpha=0.1)
ax.plot(cutleagueavg['Year'], cutleagueavg['League Two'], color='brown')
ax.fill_between(cutleagueavg['Year'], cutleagueavg['League Two']-cutleagueavg['L2Std'],cutleagueavg['League Two']+cutleagueavg['L2Std'], facecolor='brown', alpha=0.1)
ax.legend(loc='upper left')
fig.suptitle('Average Elo Score Within Each Division', fontsize=18)
ax.set_xlabel('Year', fontsize=16)
ax.set_ylabel('Elo Score', fontsize=16)
#plt.show()

plt.savefig('AverageElo.png')

In [203]:
rows2=[]
for i in range(len(years)):
    sortedp=sorted(years[i][1])
    lows=np.mean(sortedp[0:3])
    highs=(np.sum(sortedp[-3:-1])+sortedp[-1])/3
    rows2.append([years[i][0], lows, highs])
    

In [204]:
highs

1917.2878832878494

In [207]:
columns=['Year', 'Bottom 3', 'Top 3']
disparity=pd.DataFrame(rows2, columns=columns)
disparity=disparity[disparity['Year']>1991].reset_index()


,index,Year,Bottom 3,Top 3
0,34,1992,1648.504447,1808.325135
1,35,1993,1650.307442,1832.896943
2,36,1994,1654.353484,1858.204742
3,37,1995,1640.073721,1859.939010
4,38,1996,1653.545654,1860.240550
5,39,1997,1688.548369,1861.542685
6,40,1998,1653.870459,1885.692898
7,41,1999,1668.352281,1896.759359
8,42,2000,1700.910605,1890.114712
9,43,2001,1681.312490,1925.599002


In [222]:
plt.plot(disparity['Year'], disparity['Bottom 3'], color='red')
plt.plot(disparity['Year'], disparity['Top 3'], color='blue')
plt.title('Average Elo Rating Of Top 3 and Bottom 3 Teams In The Premier League', fontsize=16)
plt.xlabel('Year', fontsize=14)
plt.ylabel('Elo Rating', fontsize=14)
plt.legend(('Bottom 3', 'Top 3'), fontsize=12)
plt.savefig('Disparity.png')

In [274]:
disparity['Difference']=disparity['Top 3']-disparity['Bottom 3']
disparity

,index,Year,Bottom 3,Top 3,Difference
0,34,1992,1648.504447,1808.325135,159.820688
1,35,1993,1650.307442,1832.896943,182.589500
2,36,1994,1654.353484,1858.204742,203.851258
3,37,1995,1640.073721,1859.939010,219.865289
4,38,1996,1653.545654,1860.240550,206.694896
5,39,1997,1688.548369,1861.542685,172.994316
6,40,1998,1653.870459,1885.692898,231.822439
7,41,1999,1668.352281,1896.759359,228.407077
8,42,2000,1700.910605,1890.114712,189.204107
9,43,2001,1681.312490,1925.599002,244.286511


In [275]:
1/(1+10**(-346.537966
/400))

0.8802534958686963

In [227]:
1/(1+10**(-182.613715/400))

0.7410070013818516

In [309]:
topthree=[[82, 78, 71], [75, 68, 68],[78, 77, 65], [79, 78, 75], [91,73,69], [80, 70, 69] ,
         [87, 80, 77], [83, 78, 69],[90, 79, 75],[95, 83, 77], [91, 83, 82], [89, 83, 68] ,
         [87, 85, 83], [90, 86, 83], [86, 85, 75], [80, 71, 71], [89, 89, 70], [89, 78, 75],
         [86, 84, 82], [87, 79, 75], [81, 71, 70]]
bottomthree=[[38, 33, 29], [40, 39, 34], [40, 35, 33], [36, 35, 30], [33, 31, 24], [34, 34, 26],
            [36, 30, 28], [42, 26, 19], [33, 33, 33], [33, 33, 32], [34, 30, 15], [38, 34, 28],
            [36, 35, 11], [34, 32, 32],[30, 30, 28],  [39, 39, 33], [36, 31, 25],[36, 28, 25],
            [33, 32, 30], [35, 33, 30], [37, 34, 17]]
year=list(range(1995, 2016, 1))

In [310]:
len(list(range(1995, 2017, 1)))

22

In [311]:
len(topthree)

21

In [312]:
len(bottomthree)

21

In [313]:
tops=[]
bottoms=[]
for i in range(len(year)):
    tops.append(np.mean(topthree[i]))
    bottoms.append(np.mean(bottomthree[i]))

In [326]:
plt.plot(year, tops, color='red')
plt.plot(year, bottoms, color='blue')
plt.xticks(np.arange(min(year), max(year)+1, 3.0))
plt.xlabel('Year', fontsize=14)
plt.ylabel('Average Points', fontsize=14)
plt.title('Average Points By Top Three And Bottom Three Teams', fontsize=16)
plt.legend(('Top 3', 'Bottom 3'), fontsize=12)
plt.savefig('AvgPoints.png')

In [327]:
bottoms

[33.333333333333336,
 37.666666666666664,
 36.0,
 33.666666666666664,
 29.333333333333332,
 31.333333333333332,
 31.333333333333332,
 29.0,
 33.0,
 32.666666666666664,
 26.333333333333332,
 33.333333333333336,
 27.333333333333332,
 32.666666666666664,
 29.333333333333332,
 37.0,
 30.666666666666668,
 29.666666666666668,
 31.666666666666668,
 32.666666666666664,
 29.333333333333332]

In [335]:
import time


def lik(E, R):
    if(R==1):
        return np.log(E)
    elif (R==.5):
        return np.log(np.sqrt(E*(1-E)))
    elif (R==0):
        return np.log(1-E)
def Eloperiods(params):
    ratingperiodlength=7.0
    k=params['k']
    h=80
    hk=params['hk']
    #minrd=params['minrd']
    curtime=time.time()
    rows=len(fulldf)
    logliks=0.0
    hburn=10
    burn=25
    startyear=fulldf['Season'].values[0]
    startday=fulldf['Season']
    Elodictionary2=copy.deepcopy(initdictionary)
    currentyear=fulldf['Season'].values[0]
    startdate=fulldf['Date'][0]
    GameCounter=0.
    Rs=0.
    Es=0.
    year=copy.deepcopy(startyear)
    totaltime=fulldf['Day#'][rows-1]-fulldf['Day#'][0]
    day=0.
    Poismatches=[]
    while day<=totaltime:
        #print(Elodictionary2)
        ratingperioddf=fulldf[(fulldf['Day#'] >=day) & (fulldf['Day#']<day+ratingperiodlength)].reset_index()
        if len(ratingperioddf!=0):

            Rs=0.
            Es=0.

            for ind, row in ratingperioddf.iterrows():
                #print('sup')
                Home=row['home']
                Away=row['visitor']
                HomeGoals=row['hgoal']
                AwayGoals=row['vgoal']
                year=row['Season']
                date=row['Date']
                tier=row['tier']
                HomeRating=Elodictionary2[Home]
                AwayRating=Elodictionary2[Away]
                Expect=E(HomeRating, AwayRating, h)
                HRes=Res(HomeGoals, AwayGoals)
                PE=(HRes-Expect)*k
                Elodictionary2[Home]=HomeRating+PE
                Elodictionary2[Away]=AwayRating-PE
                Rs+=HRes
                Es+=Expect
                if(year>startyear+burn):
                    logliks+=lik(Expect, HRes)
                if(year>2008):
                    if(tier==1):
                        Poismatches.append([Home, Away,year,  HomeGoals, AwayGoals, HomeRating, AwayRating, h, 1])
                        Poismatches.append([Away, Home,year,  AwayGoals, HomeGoals, AwayRating, HomeRating, h, -1])
                        
                #print(Home,Away, HomeRating, AwayRating, PE, HRes, HomeGoals, AwayGoals)
                #print(Elodictionary2['Home'], Elodictionary2['Away'], h)
                    
            
            if(year>startyear+hburn):
                h+=hk*(Rs-Es)
        day+=ratingperiodlength
    print(params ,logliks,Elodictionary2['Tottenham Hotspur'], time.time()-curtime)
    return Poismatches

In [336]:
Poissonrows=Eloperiods(best)
columns=['Team', 'Opponent','Year', 'Goals', 'OppGoals', 'Rating', 'OppRating', 'HF', 'HomeAway']
EloPoissonDf=pd.DataFrame(Poissonrows, columns=columns)

{'hk': 0.2742633382088742, 'k': 13.797215454070143} -44210.7270572 1960.1077540754864 20.993250131607056


In [339]:
EloPoissonDf

,Team,Opponent,Year,Goals,OppGoals,Rating,OppRating,HF,HomeAway
0,Portsmouth,Fulham,2009,0,1,1729.393688,1750.271637,52.702605,1
1,Fulham,Portsmouth,2009,1,0,1750.271637,1729.393688,52.702605,-1
2,Bolton Wanderers,Sunderland,2009,0,1,1713.619525,1671.675425,52.702605,1
3,Sunderland,Bolton Wanderers,2009,1,0,1671.675425,1713.619525,52.702605,-1
4,Blackburn Rovers,Manchester City,2009,0,2,1753.738406,1747.577803,52.702605,1
5,Manchester City,Blackburn Rovers,2009,2,0,1747.577803,1753.738406,52.702605,-1
6,Stoke City,Burnley,2009,2,0,1699.737448,1635.508728,52.702605,1
7,Burnley,Stoke City,2009,0,2,1635.508728,1699.737448,52.702605,-1
8,Everton,Arsenal,2009,1,6,1844.242480,1934.351088,52.702605,1
9,Arsenal,Everton,2009,6,1,1934.351088,1844.242480,52.702605,-1


In [340]:
EloPoissonDf.to_csv('EloPoisson.csv')

In [32]:
import time


def lik(E, R):
    if(R==1):
        return np.log(E)
    elif (R==.5):
        return np.log(np.sqrt(E*(1-E)))
    elif (R==0):
        return np.log(1-E)
def Eloweirdh(params):
    ratingperiodlength=7.0
    k=params['k']
    h=[80,80,80,80]
    hk=params['hk']
    #minrd=params['minrd']
    curtime=time.time()
    rows=len(fulldf)
    logliks=0.0
    hburn=9
    burn=41
    startyear=fulldf['Season'].values[0]
    startday=fulldf['Season']
    Elodictionary2=copy.deepcopy(initdictionary)
    currentyear=fulldf['Season'].values[0]
    startdate=fulldf['Date'][0]
    GameCounter=0.
    Rs=[0,0,0,0]
    Es=[0,0,0,0]
    year=copy.deepcopy(startyear)
    totaltime=fulldf['Day#'][rows-1]-fulldf['Day#'][0]
    day=0.
    games=0
    while day<=totaltime:
        #print(Elodictionary2)
        ratingperioddf=fulldf[(fulldf['Day#'] >=day) & (fulldf['Day#']<day+ratingperiodlength)].reset_index()
        if len(ratingperioddf!=0):

            Rs=[0,0,0,0]
            Es=[0,0,0,0]

            for ind, row in ratingperioddf.iterrows():
                Home=row['home']
                Away=row['visitor']
                HomeGoals=row['hgoal']
                AwayGoals=row['vgoal']
                year=row['Season']
                date=row['Date']
                tier=row['tier']
                HomeRating=Elodictionary2[Home]
                AwayRating=Elodictionary2[Away]
                Expect=E(HomeRating, AwayRating, h[tier-1])
                HRes=Res(HomeGoals, AwayGoals)
                PE=(HRes-Expect)*k
                Elodictionary2[Home]=HomeRating+PE
                Elodictionary2[Away]=AwayRating-PE
            
                Rs[tier-1]+=HRes
                Es[tier-1]+=Expect
                if(year>startyear+burn):
                    if(tier==1):
                        logliks+=lik(Expect, HRes)
                        games+=1
                #print(Home,Away, HomeRating, AwayRating, PE, HRes, HomeGoals, AwayGoals)
                #print(Elodictionary2['Home'], Elodictionary2['Away'], h)
                    
            
            if(year>startyear+hburn):
                for i in range(4):
                    h[i]+=hk*(Rs[i]-Es[i])
        day+=ratingperiodlength
    print(params ,logliks,Elodictionary2['Tottenham Hotspur'], h,games,time.time()-curtime)
    return -1*logliks

In [35]:
from hyperopt import hp
space = {
    #'ratingperiodlength': hp.quniform('ratingperiodlength', 2, 30, 1),
    'c': hp.uniform('c', 0, 40),
    #'h': hp.uniform('h', 50, 150),
    #get rid of this for now
    #'minrd': hp.uniform('minrd', 0, 60),
    'hk': hp.uniform('hk', 0, .25),
    'maxrd': hp.uniform('maxrd', 100, 350)
}

from hyperopt import tpe
# Algorithm
tpe_algorithm = tpe.suggest

from hyperopt import Trials
# Trials object to track progress
bayes_trials = Trials()

from hyperopt import fmin

MAX_EVALS = 1000



space = {
        #'ratingperiodlength': hp.quniform('ratingperiodlength', 2, 30, 1),
        'k': hp.uniform('k', 0, 50),
        #'h': hp.uniform('h', 50, 150),
        #get rid of this for now
        #'minrd': hp.uniform('minrd', 0, 60),
        'hk': hp.uniform('hk', 0, .5),
    }

from hyperopt import tpe
    # Algorithm
tpe_algorithm = tpe.suggest

from hyperopt import Trials
    # Trials object to track progress
bayes_trials = Trials()

from hyperopt import fmin

MAX_EVALS = 1000

bestnewh = fmin(fn = Eloweirdh, space = space, algo = tpe.suggest, 
        max_evals = MAX_EVALS, trials = bayes_trials)

{'hk': 0.48705346847242026, 'k': 9.780339736023326} -3988.1019229 1898.3196927199658 [68.60198905655324, 53.84164610622284, 48.72619953837083, 28.48109355698306] 6460 26.783798933029175
{'hk': 0.3002922101017358, 'k': 29.080789298290505} -4007.57777993 2108.8961323132858 [73.28252761847419, 56.604837610052556, 50.75908594614541, 34.124502374033874] 6460 23.312000036239624
{'hk': 0.28391596854457074, 'k': 44.64455382869772} -4046.83329486 2184.5860511731694 [75.0350585486753, 58.13648619315319, 52.12476826664429, 35.473411565897905] 6460 22.10821294784546
{'hk': 0.32942560699232165, 'k': 37.96128117127649} -4028.86094905 2156.607674888959 [73.63808349003668, 57.16904912696985, 51.13713114667696, 33.34491511276165] 6460 20.97537899017334
{'hk': 0.448152023017525, 'k': 2.2171320216040566} -4051.9039339 1663.9816938466352 [65.1841224526217, 52.29978002646699, 47.670352941781005, 28.71554841412086] 6460 20.444717168807983
{'hk': 0.17903310120876187, 'k': 16.23590332564212} -3986.88679842 19

{'hk': 0.4939139098884463, 'k': 2.6413383391969845} -4040.5571009 1686.3537532795146 [65.3649331142852, 52.497502685988344, 47.912176220460665, 27.997210206346093] 6460 19.5231192111969
{'hk': 0.43984060887374293, 'k': 40.25234308437771} -4034.87850716 2166.0313426890493 [72.38316321528379, 57.22492127487126, 51.12454149460814, 30.390268202341485] 6460 19.510426998138428
{'hk': 0.36695664151327795, 'k': 13.785590800432686} -3985.56186415 1963.762707610266 [70.50045166923746, 54.39950093996266, 48.967948728385174, 31.18959315031667] 6460 19.584613800048828
{'hk': 0.36671393562954735, 'k': 4.235557415415409} -4014.77112345 1755.0390101984926 [67.84276911045255, 52.74056907852432, 47.94324660320129, 30.671372092787365] 6460 19.793099880218506
{'hk': 0.2584833462370122, 'k': 8.73881619377983} -3989.99075085 1878.1762659507863 [71.23012363843701, 54.622535282656784, 49.57624027017768, 34.998133386117175] 6460 23.364105224609375
{'hk': 0.17432844408487402, 'k': 17.10682396102991} -3987.66070

{'hk': 0.28506254813405135, 'k': 7.599062491165769} -3993.26133695 1852.9826575166383 [70.43109385537176, 54.006847858753645, 48.99372303446013, 33.67317449732099] 6460 21.02623224258423
{'hk': 0.37195294308651694, 'k': 13.681859770757697} -3985.55053161 1962.2819767878698 [70.42190088514137, 54.37017462149129, 48.94357613649199, 31.047972486951707] 6460 21.50945520401001
{'hk': 0.4022743193504262, 'k': 16.706800234644433} -3987.14548716 2001.2218776998632 [70.47009718017047, 54.75247010340652, 49.19273696752867, 30.41109608032801] 6460 19.752015829086304
{'hk': 0.4344734711119376, 'k': 5.0116263587526415} -4007.46615427 1781.8491108511673 [67.59110242708309, 52.82367344042012, 48.007938555541955, 29.18638263840621] 6460 20.340986013412476
{'hk': 0.464376524543429, 'k': 31.45400359502374} -4012.95980076 2122.445397402623 [71.34134012293612, 56.51114064142289, 50.56635312192218, 29.62684359111775] 6460 20.929892778396606
{'hk': 0.3042957201816723, 'k': 26.19632163803424} -4001.5426261 2

{'hk': 0.27320849208759185, 'k': 12.046449636398053} -3985.74849464 1938.0966635111627 [71.64895620467276, 54.91340733630567, 49.65179912579675, 34.488213401127666] 6460 19.391510009765625
{'hk': 0.19862526847845435, 'k': 34.319580064773405} -4019.92793556 2140.203713398524 [76.15310863523435, 59.29000094153628, 53.75806327008477, 40.47383592449103] 6460 19.398520946502686
{'hk': 0.24370803344843397, 'k': 19.451015023526132} -3990.15076743 2031.9110566320583 [73.33932623523204, 56.45114575870732, 50.975301216504725, 36.47082552531531] 6460 19.74629306793213
{'hk': 0.31869815942767976, 'k': 18.37796520282965} -3988.79816585 2020.2931814955361 [71.80245505357212, 55.27340922361979, 49.67659165384947, 32.93775536114324] 6460 19.819061994552612
{'hk': 0.22870570515519173, 'k': 16.689460330125197} -3987.11954461 2001.843277983639 [73.32457327425037, 56.46747245942577, 51.12946728768181, 37.241164387795685] 6460 19.67511796951294
{'hk': 0.2903532576327577, 'k': 36.098553090803264} -4024.1356

{'hk': 0.33720431438044846, 'k': 10.512304026973023} -3986.97619473 1912.0690514682544 [70.33868793879272, 54.024767567302824, 48.77907096992238, 31.916811163223844] 6460 19.578560829162598
{'hk': 0.2848245569946702, 'k': 4.554233064412138} -4011.40970468 1766.597470104123 [69.1780393798836, 53.4318071897813, 48.64453703315442, 33.44131471016346] 6460 19.771650075912476
{'hk': 0.22228892885144713, 'k': 17.21538901039113} -3987.61147055 2008.0096000760154 [73.5579288600132, 56.710676836890386, 51.37955808605944, 37.70367153408852] 6460 19.601963996887207
{'hk': 0.4629504033391899, 'k': 14.29583702720528} -3985.74859977 1970.5698845586712 [69.52799026399653, 54.49358833093166, 49.088509252823314, 29.062696016286573] 6460 19.52000880241394
{'hk': 0.2686574456541244, 'k': 7.714937058015219} -3992.86603913 1855.7172910682577 [70.755843402051, 54.266964939432924, 49.26964904347465, 34.42237474648354] 6460 19.593020915985107
{'hk': 0.3936532735582665, 'k': 1.2991620337254357} -4093.10910433 1

...

{'hk': 0.3024601257852788, 'k': 13.40850748578788} -3985.49192756 1958.608972270393 [71.37038346996228, 54.74915153893061, 49.37607568577511, 33.30459744686846] 6460 19.03805899620056
{'hk': 0.2904939225810517, 'k': 11.312684753197482} -3986.1890194 1926.0537083435388 [71.20760191729413, 54.564792534218746, 49.3143133999468, 33.67531796886013] 6460 18.96768307685852
{'hk': 0.25774199143032617, 'k': 15.301179178518142} -3986.09269671 1984.5754895370335 [72.47581697182522, 55.64069260571277, 50.26613828246159, 35.45182117095243] 6460 19.037270069122314
{'hk': 0.27365818623842075, 'k': 16.706449178821202} -3987.09740761 2001.7534490167925 [72.3608125500208, 55.5627223674276, 50.09811509208484, 34.731538339975394] 6460 18.92673087120056
{'hk': 0.23233733039353696, 'k': 9.706421033931377} -3988.09099617 1897.5707372795353 [72.02053486113901, 55.350667009956844, 50.30859974301367, 36.542801194905586] 6460 18.96474599838257
{'hk': 0.30932817014999114, 'k': 12.702928561633104} -3985.54665925 1

{'hk': 0.3402856335372425, 'k': 18.757173975317667} -3989.25556275 2024.2112346261583 [71.52405185046956, 55.17930797344485, 49.549503395650476, 32.21372426771955] 6460 18.99241328239441
{'hk': 0.2858996231038802, 'k': 15.662273254729513} -3986.30399474 1989.0015685061346 [71.99042863628375, 55.2548584601102, 49.807102941706276, 34.114803253785965] 6460 18.776221752166748
{'hk': 0.3237988223698681, 'k': 16.383008280739496} -3986.83310595 1997.6639194785257 [71.46669635752531, 54.978051814829804, 49.45131158302705, 32.660376020226735] 6460 18.894136667251587
{'hk': 0.2628069339237314, 'k': 13.91855577994004} -3985.55415081 1966.039063919623 [72.1628891790811, 55.35798379392411, 50.029061446559716, 35.108429798817845] 6460 18.855543851852417
{'hk': 0.27547390829929774, 'k': 12.458559656886287} -3985.59895047 1944.538809517185 [71.67969039960637, 54.94196349794815, 49.65563453667197, 34.40669835854961] 6460 18.78286600112915
{'hk': 0.2978765931097227, 'k': 6.244147723263597} -3999.1374199

{'hk': 0.05670457680022248, 'k': 11.719858739095818} -3987.23163756 1934.7300044155245 [80.15086199228779, 70.58799507896951, 67.98784365653006, 63.53160301574306] 6460 23.2253360748291
{'hk': 0.29914890424034446, 'k': 16.488285512046993} -3986.91206773 1999.0347500185615 [71.8753039810336, 55.210528813588276, 49.70895078191154, 33.59776010577168] 6460 21.337398052215576
{'hk': 0.21754634002520673, 'k': 7.92041906082922} -3992.23702547 1860.589515880703 [71.91860888523038, 55.45007942996963, 50.56002672322273, 37.369007718232176] 6460 22.72650694847107
{'hk': 0.22725747401094754, 'k': 12.971027246344862} -3985.53140452 1952.559029391263 [72.78403108613992, 55.99275464515414, 50.80305201169484, 37.09615354036322] 6460 22.3027400970459
{'hk': 0.2553475085080123, 'k': 4.751509236131675} -4009.55753634 1773.4478014132594 [69.82684708991107, 53.93934156855696, 49.18049622994577, 34.83232326020227] 6460 24.005537033081055
{'hk': 0.3105573830018667, 'k': 18.12391256009318} -3988.50735298 2017

{'hk': 0.26920876968158375, 'k': 10.25971445614049} -3987.26253769 1907.7356042970653 [71.37671714723925, 54.69393010719312, 49.535937504921904, 34.567942053917584] 6460 22.87363886833191
{'hk': 0.30516200431532337, 'k': 17.162966870811786} -3987.51134378 2006.8958981194976 [71.86496551301347, 55.237532209405494, 49.701711064410425, 33.391342034448904] 6460 22.174500942230225
{'hk': 0.19042328774179068, 'k': 13.434920747781716} -3985.60613483 1959.6319758104626 [73.8771217353348, 57.30150925824714, 52.24879982512278, 39.92303675404367] 6460 21.995676279067993
{'hk': 0.23237615794641814, 'k': 8.071363021521329} -3991.76388932 1863.9125649453824 [71.6047448173469, 55.06886402809244, 50.125404675461446, 36.415115985294406] 6460 21.631744146347046
{'hk': 0.28151944867798323, 'k': 5.772521403945105} -4001.86781247 1805.4680447653498 [69.84067319932016, 53.71376784396515, 48.83848363873428, 33.68926627147416] 6460 21.428446054458618
{'hk': 0.25378875131580797, 'k': 14.680316524807866} -3985.

{'hk': 0.2570778769155793, 'k': 12.618755081498982} -3985.56241323 1947.0820505474592 [72.0625468787538, 55.271220669009324, 50.014582103053314, 35.325901719067545] 6460 21.56810474395752
{'hk': 0.3012341936358056, 'k': 16.69664960373207} -3987.08777179 2001.4946349505803 [71.86836047148813, 55.21612310028457, 49.70347986829158, 33.52406590274913] 6460 21.524173974990845
{'hk': 0.285571882354854, 'k': 9.739962138056015} -3988.02169117 1898.0020277469973 [70.97083833388619, 54.371890687091536, 49.21858185516804, 33.790725002663116] 6460 21.41212010383606
{'hk': 0.2514910182268304, 'k': 15.64182639851318} -3986.30223317 1988.9340783692332 [72.6560705157307, 55.80628583269668, 50.43463507293712, 35.809316287549215] 6460 21.536607027053833
{'hk': 0.32828223894843633, 'k': 19.292937086851953} -3989.92360646 2029.8010702378388 [71.7664005526986, 55.316945737400694, 49.67857774131832, 32.639537907439816] 6460 21.640179872512817
{'hk': 0.2417372184323608, 'k': 13.982299820891427} -3985.580516 

In [36]:
bestnewh

{'hk': 0.2842430457811996, 'k': 13.278592354933826}

In [37]:
Eloweirdh(bestnewh)

{'hk': 0.2842430457811996, 'k': 13.278592354933826} -3985.48593802 1956.799228886807 [71.65899570090666, 54.94161335894927, 49.60067858294828, 34.05700531771438] 6460 25.77629327774048


3985.4859380191001